In [171]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [172]:
data = pd.read_json("C:/Users/Meyke/Documents/Projecten/Vendee/0_85af9c3ad0944b8aac9dfac7666b8674_1.json", orient="records", lines=True)

In [173]:
data.tail(10)

,boat,latitude,longitude,heading,speed,time,EventProcessedUtcTime,PartitionId,EventEnqueuedUtcTime
3320,0,37.747979,-10.537711,238.834699,14.643653,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3321,1,38.256729,-9.827544,226.661457,9.378078,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3322,2,38.274652,-9.844028,215.303118,5.070161,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3323,3,37.843140,-10.357326,212.339294,2.151606,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3324,4,38.053836,-10.121600,240.672691,7.652952,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3325,5,38.330311,-9.736073,213.938856,17.527448,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3326,6,38.270832,-9.899322,224.505695,13.911123,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3327,7,37.689256,-10.625974,212.312824,15.266012,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3328,8,37.711334,-10.595991,227.913756,11.733489,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3329,9,37.594073,-10.750392,228.470856,10.111671,1630608658,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z


In [174]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3330 entries, 0 to 3329
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   boat                   3330 non-null   int64  
 1   latitude               3330 non-null   float64
 2   longitude              3330 non-null   float64
 3   heading                3330 non-null   float64
 4   speed                  3330 non-null   float64
 5   time                   3330 non-null   int64  
 6   EventProcessedUtcTime  3330 non-null   object 
 7   PartitionId            3330 non-null   int64  
 8   EventEnqueuedUtcTime   3330 non-null   object 
dtypes: float64(4), int64(3), object(2)
memory usage: 234.3+ KB


In [175]:
time = data['time'].unique()

In [176]:
TotalDistanceTravelled = {}

# Calculate total distance travelled
for time in data.time.unique():
    TotalDistanceTravelled[time] = {}
    for boat in data.boat.unique():
        #radius of the Earth
        R = 6373.0
        
        dataN = data[data['time'] == time]

        #coordinates
        lat1 = math.radians(38.6241832)
        lon1 = math.radians(-9.3925219)
        lat2 = math.radians(dataN[dataN['boat'] == boat].iloc[-1,1])
        lon2 = math.radians(dataN[dataN['boat'] == boat].iloc[-1,2])

        #change in coordinates
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        #change in coordinates
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2

        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c

        TotalDistanceTravelled[time][boat] = distance

In [177]:
TotalTravelled2 = pd.DataFrame.from_dict({(i,j): TotalDistanceTravelled[i][j] 
                           for i in TotalDistanceTravelled.keys() 
                           for j in TotalDistanceTravelled[i].keys()},
                       orient='index')

In [178]:
TotalTravelled2.head(10)

,0
"(1630568818, 0)",0.813809
"(1630568818, 1)",1.124782
"(1630568818, 2)",0.234407
"(1630568818, 3)",0.927606
"(1630568818, 4)",1.427177
"(1630568818, 5)",1.058820
"(1630568818, 6)",1.053920
"(1630568818, 7)",0.658712
"(1630568818, 8)",0.919800
"(1630568818, 9)",0.405287


In [179]:
TotalTravelled2 = TotalTravelled2.reset_index()

In [180]:
TotalTravelled2 = TotalTravelled2.rename(columns={'index': 'Time-Boat', 0:'Distance Travelled'})

In [181]:
TotalTravelled2['time'] = TotalTravelled2.apply(lambda row: row['Time-Boat'][0], axis=1)

In [182]:
TotalTravelled2['boat'] = TotalTravelled2.apply(lambda row: row['Time-Boat'][1], axis=1)

In [183]:
del TotalTravelled2['Time-Boat']

In [184]:
RankDistanceTravelled = {}

# Calculate total distance travelled
for time in data.time.unique():
    RankDistanceTravelled[time] = {}
    TotalTravelled2N = TotalTravelled2[TotalTravelled2['time'] == time]
    listDistances = TotalTravelled2N['Distance Travelled'].to_list()
    listDistances.sort(reverse=True)
    for distance in listDistances:
        RankDistanceTravelled[time][distance] = listDistances.index(distance) + 1

In [185]:
TotalTravelled3 = pd.DataFrame.from_dict({(i,j): RankDistanceTravelled[i][j] 
                           for i in RankDistanceTravelled.keys() 
                           for j in RankDistanceTravelled[i].keys()},
                       orient='index')

In [186]:
TotalTravelled3 = TotalTravelled3.reset_index()
TotalTravelled3 = TotalTravelled3.rename(columns={'index': 'Time-Distance', 0:'Rank'})

TotalTravelled3['time'] = TotalTravelled3.apply(lambda row: row['Time-Distance'][0], axis=1)
TotalTravelled3['Distance Travelled'] = TotalTravelled3.apply(lambda row: row['Time-Distance'][1], axis=1)

In [187]:
del TotalTravelled3['Time-Distance']

In [188]:
TotalTravelled3.head(10)

,Rank,time,Distance Travelled
0,1,1630568818,1.427177
1,2,1630568818,1.124782
2,3,1630568818,1.058820
3,4,1630568818,1.053920
4,5,1630568818,0.927606
5,6,1630568818,0.919800
6,7,1630568818,0.813809
7,8,1630568818,0.658712
8,9,1630568818,0.405287
9,10,1630568818,0.234407


In [189]:
TotalTravelled = TotalTravelled3.merge(TotalTravelled2, on=['time','Distance Travelled'])

In [190]:
TotalTravelled = TotalTravelled.merge(data, on=['time','boat'])

In [191]:
TotalTravelled

,Rank,time,Distance Travelled,boat,latitude,longitude,heading,speed,EventProcessedUtcTime,PartitionId,EventEnqueuedUtcTime
0,1,1630568818,1.427177,4,38.614716,-9.403606,230.835196,8.755974,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z
1,2,1630568818,1.124782,1,38.614134,-9.393962,243.595338,5.663308,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z
2,3,1630568818,1.058820,5,38.614806,-9.390425,223.677814,4.822775,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z
3,4,1630568818,1.053920,6,38.632397,-9.386475,219.383976,10.392028,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z
4,5,1630568818,0.927606,3,38.630527,-9.399451,223.908281,13.463145,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z
...,...,...,...,...,...,...,...,...,...,...,...
3325,6,1630608658,89.835278,4,38.053836,-10.121600,240.672691,7.652952,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3326,7,1630608658,59.108595,6,38.270832,-9.899322,224.505695,13.911123,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3327,8,1630608658,55.739380,1,38.256729,-9.827544,226.661457,9.378078,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z
3328,9,1630608658,55.303027,2,38.274652,-9.844028,215.303118,5.070161,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z


In [192]:
DistanceFrontRunner = {}

# Compute distance between boats and front-runner
for time in data.time.unique():
    DistanceFrontRunner[time] = {}
    TotalTravelledXN = TotalTravelled[TotalTravelled['time'] == time]
    
    # Get FrontRunner Boat
    FrontRunner = TotalTravelledXN[TotalTravelledXN['Rank']==1].iloc[0,3]
    # Get Long & Lat FrontRunner
    FrLong = TotalTravelledXN[TotalTravelledXN['boat'] == FrontRunner].iloc[-1,5]
    FrLat = TotalTravelledXN[TotalTravelledXN['boat'] == FrontRunner].iloc[-1,4]
    
    DistanceFrontRunner[time] = {}
    
    # Calculate total distance travelled

    for boat in data.boat.unique():
        #radius of the Earth
        R = 6373.0

        #coordinates
        lat1 = math.radians(FrLat)
        lon1 = math.radians(FrLong)
        lat2 = math.radians(TotalTravelledXN[TotalTravelledXN['boat'] == boat].iloc[-1,4])
        lon2 = math.radians(TotalTravelledXN[TotalTravelledXN['boat'] == boat].iloc[-1,5])

        #change in coordinates
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        #change in coordinates
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2

        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c

        DistanceFrontRunner[time][boat] = distance

In [193]:
TotalTravelled4 = pd.DataFrame.from_dict({(i,j): DistanceFrontRunner[i][j] 
                           for i in DistanceFrontRunner.keys() 
                           for j in DistanceFrontRunner[i].keys()},
                       orient='index')

In [194]:
TotalTravelled4

,0
"(1630568818, 0)",2.224748
"(1630568818, 1)",0.840667
"(1630568818, 2)",1.660958
"(1630568818, 3)",1.795394
"(1630568818, 4)",0.000000
...,...
"(1630608658, 5)",120.905551
"(1630608658, 6)",106.024156
"(1630608658, 7)",15.237231
"(1630608658, 8)",18.841403


In [195]:
TotalTravelled4 = TotalTravelled4.reset_index()
TotalTravelled4 = TotalTravelled4.rename(columns={'index': 'Time-Boat', 0:'Distance To Front Runner'})

TotalTravelled4['time'] = TotalTravelled4.apply(lambda row: row['Time-Boat'][0], axis=1)
TotalTravelled4['boat'] = TotalTravelled4.apply(lambda row: row['Time-Boat'][1], axis=1)

In [196]:
del TotalTravelled4['Time-Boat']

In [197]:
TotalTravelled = TotalTravelled.merge(TotalTravelled4, on=['time','boat'])

In [198]:
TotalTravelled

,Rank,time,Distance Travelled,boat,latitude,longitude,heading,speed,EventProcessedUtcTime,PartitionId,EventEnqueuedUtcTime,Distance To Front Runner
0,1,1630568818,1.427177,4,38.614716,-9.403606,230.835196,8.755974,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.000000
1,2,1630568818,1.124782,1,38.614134,-9.393962,243.595338,5.663308,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,0.840667
2,3,1630568818,1.058820,5,38.614806,-9.390425,223.677814,4.822775,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.145628
3,4,1630568818,1.053920,6,38.632397,-9.386475,219.383976,10.392028,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,2.466575
4,5,1630568818,0.927606,3,38.630527,-9.399451,223.908281,13.463145,2021-09-07T20:14:44.3587988Z,1,2021-09-07T20:14:44.2550000Z,1.795394
...,...,...,...,...,...,...,...,...,...,...,...,...
3325,6,1630608658,89.835278,4,38.053836,-10.121600,240.672691,7.652952,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,75.281285
3326,7,1630608658,59.108595,6,38.270832,-9.899322,224.505695,13.911123,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,106.024156
3327,8,1630608658,55.739380,1,38.256729,-9.827544,226.661457,9.378078,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.492635
3328,9,1630608658,55.303027,2,38.274652,-9.844028,215.303118,5.070161,2021-09-08T07:20:21.3424511Z,1,2021-09-08T07:20:21.2710000Z,109.785378


In [199]:
TotalTravelled.to_csv("C:/Users/Meyke/Documents/Projecten/Vendee/0_85af9c3ad0944b8aac9dfac7666b8674_1.csv")